In [60]:
import pandas as pd
import numpy as np
import json
import requests
import xmltodict
import time
import pickle
import os
import sqlalchemy
import urllib.request
from mutagen.mp3 import MP3

In [9]:
config = json.load(open('config.json', 'r'))

In [12]:
user = config['db']['username']
password = config['db']['password']
host = config['db']['host']
port = config['db']['port']
dbname = config['db']['dbname']

In [13]:
template = "mysql+pymysql://{user}:{password}@{host}:{port}/{dbname}"
connection_string = template.format(user=user, password=password, host=host, port=port, dbname=dbname)

In [15]:
conn = sqlalchemy.create_engine(connection_string, pool_size=1)

In [19]:
fname = 'feed.xml'
url = 'http://dataskeptic.com/feed.rss'

if not(os.path.isfile(fname)):
    print('fetching')
    r = requests.get(url)
    f = open(fname, 'wb')
    f.write(r.text.encode('utf-8'))
    f.close()

with open(fname) as fd:
    xml = xmltodict.parse(fd.read())

fetching


In [52]:
emap = {}
episodes = xml['rss']['channel']['item']
for episode in episodes:
    guid = episode['guid']['#text']
    url = episode['enclosure']['@url']
    emap[guid] = url

In [47]:
q = """
SELECT t1.blog_id, prettyname, guid
 , t2.content_id as mp3_content, t3.content_id as homepage_content
 , t1.title, t1.abstract
FROM blog t1
LEFT JOIN related_content t2
 on t1.blog_id = t2.blog_id
 and t2.type='mp3'
LEFT JOIN related_content t3
 on t1.blog_id = t3.blog_id
 and t3.type='homepage-image'
WHERE t1.prettyname like '/episodes/%%'
AND (t2.blog_id is NULL or t3.blog_id is null)
LIMIT 10
"""
df = pd.read_sql(q, conn)

In [61]:
response = urllib.request.urlopen(dest)

In [63]:
for r in range(df.shape[0]):
    row = df.iloc[r]
    blog_id = row['blog_id']
    guid = row['guid']
    title = row['title']
    body = row['abstract']
    dest = emap[guid]
    response = urllib.request.urlopen(dest)
    data = response.read()
    fname = 'temp.mp3'
    f = open(fname, 'wb')
    f.write(data)
    f.close()
    audio = MP3(fname)
    duration = int(audio.info.length)
    q1 = tpl.format(blog_id=blog_id, dest=dest, type='mp3', title=title, body=body, duration=duration)
    conn.execute(q1)
    dest = 'https://s3.amazonaws.com/dataskeptic.com/img/2018/ai-cover.png'
    q2 = tpl.format(blog_id=blog_id, dest=dest, type='homepage-image', title=title, body=body, duration=-1)
    conn.execute(q2)